In [47]:
# imports for required libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

from werkzeug.security import generate_password_hash, check_password_hash

from sqlalchemy import create_engine

In [2]:
# load datased as pandas dataframe
df = pd.read_csv('TopStaredRepositories.csv')

In [3]:
# print first 5 rows of dataset
df.head(5)

,Username,Repository Name,Description,Last Update Date,Language,Number of Stars,Tags,Url,Gravatar
0,freeCodeCamp,freeCodeCamp,The https://freeCodeCamp.com open source codeb...,2017-06-24T15:56:17Z,JavaScript,290k,"nonprofits,certification,curriculum,react,node...",https://github.com/freeCodeCamp/freeCodeCamp,https://avatars0.githubusercontent.com/u/98925...
1,twbs,bootstrap,"The most popular HTML, CSS, and JavaScript fra...",2017-06-24T15:40:21Z,JavaScript,112k,"javascript,css,html,bootstrap,jekyll-site,scss",https://github.com/twbs/bootstrap,https://avatars0.githubusercontent.com/u/29185...
2,EbookFoundation,free-programming-books,Freely available programming books,2017-06-23T01:09:34Z,NaN,87.8k,"education,list,books,resource",https://github.com/EbookFoundation/free-progra...,https://avatars0.githubusercontent.com/u/14127...
3,facebook,react,"A declarative, efficient, and flexible JavaScr...",2017-06-24T19:33:49Z,JavaScript,69.7k,NaN,https://github.com/facebook/react,https://avatars3.githubusercontent.com/u/69631...
4,d3,d3,"Bring data to life with SVG, Canvas and HTML.",2017-05-31T06:03:47Z,JavaScript,65.7k,visualization,https://github.com/d3/d3,https://avatars1.githubusercontent.com/u/15627...


In [4]:
# print number of rows and columns in dataset, 980 and 9 respectively
df.shape

(980, 9)

In [56]:
df.rename(columns={"Username": "username"}, inplace=True)
df["email"] = df["username"] + "@example.com"
df["about_me"] = "This is " + df["username"] + "'s profile"
df["last_seen"] = "2020-08-20 17:42:53.249352"
# commented below to avoid regenerating data each time cell runs
# df["password_hash"] = df["username"].apply(lambda x: generate_password_hash(x))

Check that passwords in dataframe have been hashed correctly by using the provided check_password_hash() function against a known password value

In [49]:
check_password_hash(df_important['password_hash'].iloc[0], "freeCodeCamp")

True

In [57]:
columns = ["username", "email", "password_hash", "about_me", "last_seen"]
df_important = df[columns]
df_important.head(5)

,username,email,password_hash,about_me,last_seen
0,freeCodeCamp,freeCodeCamp@example.com,pbkdf2:sha256:150000$UskQ9iRc$6e52d2ba61ac8631...,This is freeCodeCamp's profile,2020-08-20 17:42:53.249352
1,twbs,twbs@example.com,pbkdf2:sha256:150000$yVEsMNo2$26aa3e7813f5ebac...,This is twbs's profile,2020-08-20 17:42:53.249352
2,EbookFoundation,EbookFoundation@example.com,pbkdf2:sha256:150000$1uG5rXvn$160e96fd724a2d47...,This is EbookFoundation's profile,2020-08-20 17:42:53.249352
3,facebook,facebook@example.com,pbkdf2:sha256:150000$tMqTa9Bw$f2f87fb5ca933ba0...,This is facebook's profile,2020-08-20 17:42:53.249352
4,d3,d3@example.com,pbkdf2:sha256:150000$f7L19M5c$1097e4bcbf6f0b05...,This is d3's profile,2020-08-20 17:42:53.249352


# Unique Constraint
The issue with this approach is that some of the entries in the original csv file have the same user (project owner in this app context) which means that the simple approach of inserting each username into the database and linking the id of that user with the project's owner fails as the usernames are not always unique. The easiest solution for now is to create a single user which owns all of the imported projects. While not being ideal, this doesn't cause the recommendations to fail or be obscured as the model does not look at a user's own projects to perform.